In [ ]:
#hide output of cell
# %%capture


import voila
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import numpy as np
from credsTest.SewingExport import *
from credsTest.SummaExport import *;
from tkinter import *
from tkinter import messagebox
import logging;


In [ ]:

class Counter:
    def __init__(self, initial=0):
      self.value = initial
    def increment(self, amount=1):
      self.value += amount
      return self.value
    def __iter__(self, sentinal=False):
      return iter(self.increment, sentinal)
counter = Counter()  


class Pattern:
    def __init__(self, initial=False):
      self.value = initial
    def textValidate(self, last2):
      if last2[0] == last2[1]:
        if createAlert() == 'yes':
          self.value = True
        else:
           self.value = False
      return self.value

pat = Pattern()

class LastValue:
    def __init__(self):
      self._list = ['','']
    def record(self, txt,counter):
      self._list[counter % 2] = txt
      return self._list

class sheetInstance:
    def __init__(self):
        self.newSpreadsheet = {}
        self.gsheetId = ''
        self.Url = ''
        self.sheet_names = []
        self.sheetIds = []
        
    def createSheet(self,pattern,nest_type):
        if len(self.newSpreadsheet) == 0:
            df = createNestFile(pattern)
            
            if nest_type == 'Sewing Export':
                df = createCoversData(df)
            elif nest_type == 'Summa Export':
                df = createSummaData(df)
            self.newSpreadsheet = SheetsNew()
            
            self.newSpreadsheet = SheetsNew()
            self.gsheetId = self.newSpreadsheet['gsheetId']
            self.Url = self.newSpreadsheet['Url']
            self.sheetProperties = self.newSpreadsheet['sheet_names']
            self.sheet_names = [self.sheetProperties[0]['properties']['title'],self.sheetProperties[1]['properties']['title']]
            self.sheetIds = [self.sheetProperties[0]['properties']['sheetId'],self.sheetProperties[1]['properties']['sheetId']]
            
            # run this to write data to google sheets
            clearSheets(self.gsheetId,self.sheet_names[0])
            writeDataToSheetDf(self.sheet_names[0],self.gsheetId,df)
            if nest_type == 'Sewing Export':
                createSewingPivot(self.newSpreadsheet,df)
            elif nest_type.value == 'Summa Export':
                createSummaPivot(self.newSpreadsheet,df)
        elif len(self.newSpreadsheet) != 0:
            print('Error Already created spreadsheet!')
        return self.newSpreadsheet

    
summaSheet = sheetInstance()
sewingSheet = sheetInstance()
last2 = LastValue()


In [ ]:
txtHeader = widgets.HTML(value = '<h1>Nesting API</h1>')
txt1 = widgets.HTML(value = '<h2>Link to Spreadsheet: </h2>')
txt3 = widgets.HTML(value = """<h3> Duplicate Spreadsheets Warning!, Are you sure?\n!You've Already Created a spreadsheet!</h3>""")
txt2 = widgets.HTML(value = """<h4>Enter file Pattern:\n i.e if from workorder (mrp.workorder (15).csv
                    pattern -> workorder</h4>""")
txt_vbox = widgets.VBox([txt2])

In [ ]:
file1 = open('img1.jpg','rb')
file2 = open('img2.jpg','rb')
pic = file1.read()
pic2 = file2.read()
img1 = widgets.Image(
    value = pic,
    format = 'jpg',
    width = '350'
)
img2 = widgets.Image(
    value = pic2,
    format = 'jpg',
    width = '300'
)

imageList = np.array([img1,img2])
anime = np.resize(imageList,100)
def f(x):
    display(anime[x])


In [ ]:
get_file = widgets.RadioButtons(
    options=['readcsv/excel file', 'upload file'],
    description='File:',
    disabled=True
)

nest_type = widgets.Dropdown(
    options=['Sewing Export', 'Summa Export','Other','What file are you uploading'],
    value='What file are you uploading',
    description='Nest Type:',
    disabled=False,
)
output_drop = widgets.Output()

def dropdown_eventhandler(change):
    with output_drop:
        output_drop.clear_output()
        print('Preparing to Upload:',{nest_type.value})
        print('You will need to provide file name pattern as shown below')
        interact(f , x=5);
nest_type.observe(dropdown_eventhandler,names='value')

pattern = widgets.Text(
    value='',
    placeholder='Enter file Pattern: i.e if from workorder (mrp.workorder (15).csv -> workorder)',
    description='File Pattern:',
    disabled=False
)
          

newSheet = widgets.ToggleButtons(
    options=['Create New', 'Upload to Existing'],
    description='Create New Spreadsheet:',
    disabled=True,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Default', 'Provide sheet id'],
#     icons=['check'] * 3
)
filters = widgets.ToggleButtons(
    options=['Default Filters', 'User Defined'],
    description='Pivot Filters:',
    disabled=True,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Use default', 'Provide columns to filter'],
#     icons=['check'] * 3
)



In [ ]:
def createAlert():
    root = Tk()
    root.lift()
    root.attributes('-topmost',True)
#     root.after_idle(root.attributes,'-topmost',False)
    
    root.withdraw()
    root.geometry("300x200")
    # root.after(3000,root.destroy)
    a =messagebox.askquestion("Multiple Spreadsheets Warning!", "Are you sure?\n!You've Already Created a spreadsheet!")

    if a:
        root.destroy()
    return a
def check():
    return pattern.value != '' and pat.textValidate(last2.record(pattern.value,counter.value))

def loop():
    if createAlert() == 'yes':
        loop()
    else:
        print('all done')
def sheetType():
    if nest_type.value == 'Sewing Export':
#         try: 
        print('creating pivot for Sewing covers')
        sewingSheet.createSheet(pattern.value,nest_type.value);
        display(txt1,sewingSheet.Url)
            
#             summaryTable(sewingSheet.a)
#         except:
#             print('Invalid file name pattern entered')

    elif nest_type.value == 'Other':
        print('This is not supported Currently')
    elif nest_type.value == 'Summa Export':
        try: 
            print('creating pivot for Summa')
            summaSheet.createSheet(pattern.value,nest_type.value);
            display(txt1,summaSheet.Url)
#             summaryTable(summaSheet.a)
        except:
            e =  logging.exception("message")
            print(e)
            print('Invalid file name pattern entered')

def summaryTable(nest):
    time_stamp = file_time_stamp(pattern.value)['time']
    file_name_read = file_time_stamp(pattern.value)['file']
    df = pd.DataFrame({'file name': [str(file_name_read)],'file_write_date':[time_stamp]})
    add_sheets(nest['gsheetId'], 'Summary Table')
    writeDataToSheetDf('Summary Table',nest['gsheetId'],df)
        

In [ ]:
from functools import partial
createNest = widgets.Button(
    description = 'Create',
    tooltip = 'Send',
    style = {'description_width': 'initial'}
)
output = widgets.Output()
testing = 'hi'

def on_button_clicked(counter,event):
    with output:
        output.clear_output()
        counter.increment()
        can_create = check()
        print('Click Counter:',counter.value)
        if can_create:
            sheetType()
        else:
            print('Creation Canceled due to empty file name or duplicate')

 
createNest.on_click(partial(on_button_clicked,counter))
vbox_result = widgets.VBox([createNest,output])

file_type = widgets.HBox([nest_type,output_drop])
enter_field = widgets.HBox([txt2,pattern])


vbox_rest = widgets.VBox([txtHeader,file_type,get_file,enter_field,vbox_result])


In [ ]:
page = widgets.HBox([vbox_rest])
display(page)

